In [1]:
import numpy as np
from glob import glob 
import os
from garuda.ops import obb_iou

In [2]:
# old_directory = '/home/shataxi.dubey/shataxi_work/gujarat_brick_kilns/ahmedabad_10km_buffer_ground_truth_labels'
# new_directory = '/home/shataxi.dubey/shataxi_work/gujarat_brick_kilns/predict/ahmedabad_obb_v1__1120_300/ahmedabad_10km_buffer_images2/labels'

old_directory = '/home/rishabh.mondal/Brick-Kilns-project/albk_rishabh/albk_v2/YOLO_LOCALIZATION/processed_data/up_10803_v1/labels'
new_directory = '/home/rishabh.mondal/Brick-Kilns-project/albk_rishabh/albk_v2/YOLO_LOCALIZATION/predict/multi_state_train_data_v2_test_uttar_pradesh_obb_v2__1120_100/labels/'

is_old_pred = False # is_old_pred = True when old directory contains predictions otherwise False
is_new_pred = True  # is_new_pred = True when new directory contains predictions otherwise False

old_files = glob(f'{old_directory}/*')
new_files = glob(f'{new_directory}/*')

In [3]:
def drop_extra_matches(matches: np.ndarray) -> np.ndarray: #copied from supervision
    """
    Deduplicate matches. If there are multiple matches for the same true or
    predicted box, only the one with the highest IoU is kept.
    """
    if matches.shape[0] > 0:
        matches = matches[matches[:, 2].argsort()[::-1]]
        matches = matches[np.unique(matches[:, 1], return_index=True)[1]]
        matches = matches[matches[:, 2].argsort()[::-1]]
        matches = matches[np.unique(matches[:, 0], return_index=True)[1]]
    return matches

In [4]:
files_not_present_previously = []
files_with_diff_num_boxes = []
files_with_incorrect_class = []

In [5]:
for new_file in new_files:
    new_fname = os.path.basename(new_file)
    old_file = os.path.join(old_directory,new_fname)
    # check if file exists
    if os.path.exists(old_file):
        new_labels = np.loadtxt(new_file,ndmin=2)
        old_labels = np.loadtxt(old_file,ndmin=2)
        # check if number of prediction matches
        if len(new_labels) == len(old_labels):
            if is_old_pred:
                old_boxes = old_labels[:,1:-1].astype(np.float32)
            else:
                old_boxes = old_labels[:,1:].astype(np.float32)
            if is_new_pred:
                new_boxes = new_labels[:,1:-1].astype(np.float32)
            else:
                new_boxes = new_labels[:,1:].astype(np.float32)
            # print(new_boxes, old_boxes)
            iou = obb_iou(new_boxes, old_boxes)
            # print(iou)
            matched_idx = np.asarray(iou > 0.9).nonzero() 
            # print(matched_idx)
            matches = np.stack(
                (matched_idx[0], matched_idx[1], iou[matched_idx]), axis=1
            )
            matches = drop_extra_matches(matches=matches)
            # print(matches)

            for box_match in matches:
                # check if class matches
                if new_labels[int(box_match[0])][0] != old_labels[int(box_match[1])][0]:
                #     print('Box matches')
                # else:
                    # print('Box does not match. Add it for hand validation')
                    files_with_incorrect_class.append(new_file)
        else:
            # print('Box does not match. Add it for hand validation')
            files_with_diff_num_boxes.append(new_file)
    else:
        # print('Box does not match. Add it for hand validation')
        files_not_present_previously.append(new_file)
    # break

In [7]:
len(files_with_incorrect_class)

128

In [11]:
files_with_diff_num_boxes

['/home/shataxi.dubey/shataxi_work/yolo_performance_on_empty_label_files/predict/test_lucknow_fold1_obb_v3__1120_100/lucknow_fold1/labels/26.95,80.88.txt',
 '/home/shataxi.dubey/shataxi_work/yolo_performance_on_empty_label_files/predict/test_lucknow_fold1_obb_v3__1120_100/lucknow_fold1/labels/26.92,81.04.txt',
 '/home/shataxi.dubey/shataxi_work/yolo_performance_on_empty_label_files/predict/test_lucknow_fold1_obb_v3__1120_100/lucknow_fold1/labels/26.70,81.19.txt',
 '/home/shataxi.dubey/shataxi_work/yolo_performance_on_empty_label_files/predict/test_lucknow_fold1_obb_v3__1120_100/lucknow_fold1/labels/26.71,81.18.txt',
 '/home/shataxi.dubey/shataxi_work/yolo_performance_on_empty_label_files/predict/test_lucknow_fold1_obb_v3__1120_100/lucknow_fold1/labels/26.73,81.14.txt']

In [8]:
len(files_with_diff_num_boxes)

1392

In [9]:
len(files_not_present_previously)

11511

In [10]:
handvalidate_files = files_with_incorrect_class + files_with_diff_num_boxes + files_not_present_previously
print('Number of files to handvalidate: ',len(handvalidate_files))

Number of files to handvalidate:  13031


In [11]:
handvalidate_files[:5]

['/home/rishabh.mondal/Brick-Kilns-project/albk_rishabh/albk_v2/YOLO_LOCALIZATION/predict/multi_state_train_data_v2_test_uttar_pradesh_obb_v2__1120_100/labels/28.02,77.81.txt',
 '/home/rishabh.mondal/Brick-Kilns-project/albk_rishabh/albk_v2/YOLO_LOCALIZATION/predict/multi_state_train_data_v2_test_uttar_pradesh_obb_v2__1120_100/labels/26.24,81.36.txt',
 '/home/rishabh.mondal/Brick-Kilns-project/albk_rishabh/albk_v2/YOLO_LOCALIZATION/predict/multi_state_train_data_v2_test_uttar_pradesh_obb_v2__1120_100/labels/28.76,77.34.txt',
 '/home/rishabh.mondal/Brick-Kilns-project/albk_rishabh/albk_v2/YOLO_LOCALIZATION/predict/multi_state_train_data_v2_test_uttar_pradesh_obb_v2__1120_100/labels/29.16,78.48.txt',
 '/home/rishabh.mondal/Brick-Kilns-project/albk_rishabh/albk_v2/YOLO_LOCALIZATION/predict/multi_state_train_data_v2_test_uttar_pradesh_obb_v2__1120_100/labels/26.85,82.59.txt']

In [17]:
new_dir="/home/rishabh.mondal/Brick-Kilns-project/albk_rishabh/albk_v2/YOLO_LOCALIZATION/processed_data/up_predict_v2/labels"
for file_path in handvalidate_files:
    if os.path.exists(file_path):
        # Create the symlink path in the new directory
        symlink_path = os.path.join(new_dir, os.path.basename(file_path))
        try:
            # Create a symbolic link
            os.symlink(file_path, symlink_path)
            print(f'Symlink created: {symlink_path}')
        except FileExistsError:
            print(f'Symlink already exists: {symlink_path}')
        except Exception as e:
            print(f'Error creating symlink for {file_path}: {e}')
    else:
        print(f'File does not exist: {file_path}')

print(f'All hand-validated files symlinked in {new_dir}')



Symlink created: /home/rishabh.mondal/Brick-Kilns-project/albk_rishabh/albk_v2/YOLO_LOCALIZATION/processed_data/up_predict_v2/labels/28.02,77.81.txt
Symlink created: /home/rishabh.mondal/Brick-Kilns-project/albk_rishabh/albk_v2/YOLO_LOCALIZATION/processed_data/up_predict_v2/labels/26.24,81.36.txt
Symlink created: /home/rishabh.mondal/Brick-Kilns-project/albk_rishabh/albk_v2/YOLO_LOCALIZATION/processed_data/up_predict_v2/labels/28.76,77.34.txt
Symlink created: /home/rishabh.mondal/Brick-Kilns-project/albk_rishabh/albk_v2/YOLO_LOCALIZATION/processed_data/up_predict_v2/labels/29.16,78.48.txt
Symlink created: /home/rishabh.mondal/Brick-Kilns-project/albk_rishabh/albk_v2/YOLO_LOCALIZATION/processed_data/up_predict_v2/labels/26.85,82.59.txt
Symlink created: /home/rishabh.mondal/Brick-Kilns-project/albk_rishabh/albk_v2/YOLO_LOCALIZATION/processed_data/up_predict_v2/labels/25.99,83.12.txt
Symlink created: /home/rishabh.mondal/Brick-Kilns-project/albk_rishabh/albk_v2/YOLO_LOCALIZATION/processed